In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q json_repair
!pip install -q wandb
!pip install -q pydantic
!pip install -qU numpy
!pip install -U transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 10.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import wandb
wandb.login(key='')

In [ ]:
import pandas as pd
from os.path import join
from tqdm import tqdm
import json
import random
import os

from pydantic import BaseModel, Field
import torch

base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None

data_dir="/kaggle/working/"

def parse_json(text):
    """
    parse the output from llm to valid json
    """
    try:
        return json_repair.loads(text)
    except:
        return None

In [ ]:
!wget http://www.phontron.com/download/conala-corpus-v1.1.zip

--2025-11-28 15:26:53--  http://www.phontron.com/download/conala-corpus-v1.1.zip
Resolving www.phontron.com (www.phontron.com)... 173.236.247.185
Connecting to www.phontron.com (www.phontron.com)|173.236.247.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52105440 (50M) [application/zip]
Saving to: ‘conala-corpus-v1.1.zip’

conala-corpus-v1.1. 100%[===================>]  49.69M  41.0MB/s    in 1.2s    

2025-11-28 15:26:55 (41.0 MB/s) - ‘conala-corpus-v1.1.zip’ saved [52105440/52105440]



In [ ]:
!unzip /kaggle/working/conala-corpus-v1.1.zip

Archive:  /kaggle/working/conala-corpus-v1.1.zip
   creating: conala-corpus/
  inflating: conala-corpus/conala-mined.jsonl  
  inflating: conala-corpus/conala-train.json  
  inflating: conala-corpus/conala-test.json  


In [ ]:
train = pd.read_json('/kaggle/working/conala-corpus/conala-train.json')

In [ ]:
train

,intent,rewritten_intent,snippet,question_id
0,How to convert a list of multiple integers int...,Concatenate elements of a list 'x' of multiple...,"sum(d * 10 ** i for i, d in enumerate(x[::-1]))",41067960
1,How to convert a list of multiple integers int...,convert a list of integers into a single integer,"r = int(''.join(map(str, x)))",41067960
2,how to convert a datetime string back to datet...,convert a DateTime string back to a DateTime o...,datetime.strptime('2010-11-13 10:33:54.227806'...,4170655
3,Averaging the values in a dictionary based on ...,get the average of a list values for each key ...,"[(i, sum(j) / len(j)) for i, j in list(d.items...",29565452
4,zip lists in python,"zip two lists `[1, 2]` and `[3, 4]` into a lis...","zip([1, 2], [3, 4])",13704860
...,...,...,...,...
2374,"how to change [1,2,3,4] to '1234' using python","join list of numbers `[1,2,3,4] ` to string of...",""""""""""""".join([1, 2, 3, 4])",2597932
2375,Delete every non utf-8 symbols froms string,delete every non `utf-8` characters from a str...,"line = line.decode('utf-8', 'ignore').encode('...",26541968
2376,How to execute a command in the terminal from ...,execute a command `command ` in the terminal f...,os.system(command),33065588
2377,Python MySQL Parameterized Queries,MySQL execute query 'SELECT * FROM foo WHERE b...,c.execute('SELECT * FROM foo WHERE bar = %s AN...,775296


In [ ]:
import json
with open('/kaggle/working/conala-corpus/conala-train.json', 'r') as json_file:
   train_data = json.load(json_file)

with open('/kaggle/working/fine_tune_train.jsonl', 'w') as jsonl_file:
   for entry in train_data:
       jsonl_file.write(json.dumps(entry) + '\n')


In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (453/453), done.
remote: Compressing objects: 100% (346/346), done.
remote: Total 453 (delta 118), reused 336 (delta 89), pack-reused 0 (from 0)
Receiving objects: 100% (453/453), 5.15 MiB | 28.52 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 34.5 MB/s et

In [ ]:
class Intent(BaseModel):
    enhanced_intent: str = Field(..., min_length=5, max_length=300,
                             description="The enhanced version of Intent.")


class Code(BaseModel):
    Code: str = Field(..., min_length=5, max_length=300,
                             description="The Output code in Python.")

In [ ]:
print(data_dir)

/kaggle/working/


In [ ]:

save_to = join(data_dir, "train.jsonl")

ix = 0
for item in tqdm(train_data):
    with open(save_to, "a", encoding="utf8") as dest:
        dest.write(json.dumps({
            "id": ix,
            "intent": item['intent'].strip(),
            "task": "write the enhanced version of the intent into a JSON.",
            "output_scheme": json.dumps( Intent.model_json_schema(), ensure_ascii=False ),
            "response": item['rewritten_intent'],
        }, ensure_ascii=False, default=str)  + "\n" )

    ix += 1


100%|██████████| 2379/2379 [00:00<00:00, 3844.16it/s]


In [ ]:
for item in tqdm(train_data):
    with open(save_to, "a", encoding="utf8") as dest:
        dest.write(json.dumps({
            "id": ix,
            "intent": item['intent'].strip(),
            "task": "write the needed code in python into a JSON.",
            "output_scheme": json.dumps( Code.model_json_schema(), ensure_ascii=False ),
            "response": item['snippet'],
        }, ensure_ascii=False, default=str)  + "\n" )

    ix += 1


100%|██████████| 2379/2379 [00:00<00:00, 3870.51it/s]


In [ ]:
intent_system_msg="\n".join([
            "You are a technical intent rewriter preparing specifications for LLM code generation.",
            "Your rewritten intent must provide ALL information needed for an LLM to generate correct code WITHOUT additional context.",
            "Essential elements to include:",
            "1. Variable names - Give explicit names (x, s, d, lst) so the LLM knows what to use",
            "2. Data structures - Specify list, dict, set, string, tuple, etc.",
            "3. Data types - State integers, strings, floats, booleans, etc.",
            "4. Operation - Use precise verbs: concatenate, filter, transform, merge, extract, aggregate, sort",
            "5. Output format - Describe what the result should be",
            "6. Implicit details - Order preservation, empty case handling, type conversions",
            "Bad: 'combine numbers in a list'",
            "Good: 'Concatenate elements of a list 'x' of multiple integers to a single integer'",
            "Why? The good version tells the LLM:",
            "- Variable name: 'x'",
            "- Input type: list of integers",
            "- Operation: concatenate (not sum, not join with delimiter)",
            "- Output type: single integer"
        ])

code_system_msg ="\n".join([
            "You are an expert Python code generator.",
            "You will receive a technical specification describing a programming task.",
            "Generate clean, efficient Python code that exactly implements the specification.",
            "Follow these rules:",
            "1. Use the exact variable names specified in the input",
            "2. Match the data structures and types mentioned",
            "3. Implement the precise operation described",
            "4. Generate only the code expression or statement - no explanations, no comments, no markdown",
            "5. Write concise, Pythonic code using appropriate built-in functions and list comprehensions",
            "6. Handle the exact input/output format specified",
            "Do not add any introduction, explanation, or conclusion.",
            "Output only executable Python code."
        ])


llm_finetuning_data = []
data_path="/kaggle/working/train.jsonl"
for line in open(data_path, "r"):
    if line.strip()=="":
        continue

    rec =json.loads(line.strip())

    if rec['task']=="write the enhanced version of the intent into a JSON.":
        system_message = intent_system_msg
    else:
        system_message = code_system_msg

    llm_finetuning_data.append({
        "system": system_message,

        "instruction":"\n".join([
            "#Intent:",
            rec["intent"],
            "",

            "#Task:",
            rec["task"],
            "",

            "#Output Scheme:",
            rec["output_scheme"],
            "",

            "#Output Json:",
            "```json"
        ]),

        "input":"",    # the task

        "output":"\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),
            "```"
        ]),

        "history":[]
    })

random.Random(101).shuffle(llm_finetuning_data)

In [ ]:
len(llm_finetuning_data)

4758

In [ ]:
train_sample_sz = 4000

train_ds = llm_finetuning_data[:train_sample_sz]
ev_ds = llm_finetuning_data[train_sample_sz:]

os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.jsonl"), "w", encoding="utf8") as dest:
    for item in train_ds:
        dest.write(json.dumps(item, ensure_ascii=False) + "\n")
with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.jsonl"), "w", encoding="utf8") as dest:
    for item in ev_ds:
        dest.write(json.dumps(item, ensure_ascii=False) + "\n")


In [ ]:
# "code_finetune_train": {
#         "file_name": "/kaggle/working/datasets/llamafactory-finetune-data/train.jsonl",
#         "columns": {
#             "prompt": "instruction",
#             "query": "input",
#             "response": "output",
#             "system": "system",
#             "history": "history"
#         }
#     },
# "code_finetune_val": {
#         "file_name": "/kaggle/working/datasets/llamafactory-finetune-data/val.jsonl",
#         "columns": {
#             "prompt": "instruction",
#             "query": "input",
#             "response": "output",
#             "system": "system",
#             "history": "history"
#         }
#     }

In [ ]:
import json

json_path = "/kaggle/working/LLaMA-Factory/data/dataset_info.json"

# Load existing JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Your datasets to add
new_entries = {
    "code_finetune_train": {
        "file_name": "/kaggle/working/datasets/llamafactory-finetune-data/train.jsonl",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    },
    "code_finetune_val": {
        "file_name": "/kaggle/working/datasets/llamafactory-finetune-data/val.jsonl",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
}

# Update and save back
data.update(new_entries)

with open(json_path, "w") as f:
    json.dump(data, f, indent=4)

print("✨ Done! JSON updated successfully.")


✨ Done! JSON updated successfully.


In [ ]:
with open(json_path) as f:
    print(json.dumps(json.load(f), indent=4))


{
    "identity": {
        "file_name": "identity.json"
    },
    "alpaca_en_demo": {
        "file_name": "alpaca_en_demo.json"
    },
    "alpaca_zh_demo": {
        "file_name": "alpaca_zh_demo.json"
    },
    "glaive_toolcall_en_demo": {
        "file_name": "glaive_toolcall_en_demo.json",
        "formatting": "sharegpt",
        "columns": {
            "messages": "conversations",
            "tools": "tools"
        }
    },
    "glaive_toolcall_zh_demo": {
        "file_name": "glaive_toolcall_zh_demo.json",
        "formatting": "sharegpt",
        "columns": {
            "messages": "conversations",
            "tools": "tools"
        }
    },
    "mllm_demo": {
        "file_name": "mllm_demo.json",
        "formatting": "sharegpt",
        "columns": {
            "messages": "messages",
            "images": "images"
        },
        "tags": {
            "role_tag": "role",
            "content_tag": "content",
            "user_tag": "user",
            "assistan

In [ ]:
import os

os.makedirs("/kaggle/working/models", exist_ok=True)
print("Folder 'models' created!")


Folder 'models' created!


In [ ]:
%%writefile /kaggle/working/LLaMA-Factory/examples/train_lora/code_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-3B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: code_finetune_train
eval_dataset: code_finetune_val
template: qwen
cutoff_len: 3500
overwrite_cache: true
preprocessing_num_workers: 8

### output
output_dir: /kaggle/working/models
logging_steps: 10
save_steps: 1000
plot_loss: true

### train
per_device_train_batch_size: 4
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 2.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 1000

report_to: wandb
run_name: code-finetune-llamafactory

push_to_hub: true
hub_model_id: "abdelalem33/coder"
hub_private_repo: true
hub_strategy: checkpoint


Writing /kaggle/working/LLaMA-Factory/examples/train_lora/code_finetune.yaml


In [ ]:
!cd /kaggle/working/LLaMA-Factory/ && llamafactory-cli train /kaggle/working/LLaMA-Factory/examples/train_lora/code_finetune.yaml

2025-11-28 15:28:16.969566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764343697.146377     302 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764343697.196160     302 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
[INFO|2025-11-28 15:28:43] llamafactory.hparams.parser:468 >> Process rank: 0